In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch.nn.functional as F
import torch
from torch import nn
from torchsummary import summary
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
torch.cuda.is_available()

True

In [3]:
from text_recognizer.networks.transformer.layers import Decoder

In [4]:
decoder = Decoder(dim=128, depth=4, num_heads=8, ff_kwargs={}, attn_kwargs={}, cross_attend=True)

In [ ]:
decoder.cuda()

In [5]:
from text_recognizer.networks.transformer.transformer import Transformer

In [6]:
transformer_decoder = Transformer(num_tokens=90, max_seq_len=690, attn_layers=decoder, emb_dim=128, emb_dropout=0.1)

In [7]:
transformer_decoder.cuda()

Transformer(
  (attn_layers): Decoder(
    (layers): ModuleList(
      (0): ModuleList(
        (0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (1): Attention(
          (qkv_fn): Sequential(
            (0): Linear(in_features=128, out_features=24576, bias=False)
            (1): Rearrange('b n (qkv h d) -> qkv b h n d', qkv=3, h=8)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (fc): Linear(in_features=8192, out_features=128, bias=True)
        )
        (2): Residual()
      )
      (1): ModuleList(
        (0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (1): Attention(
          (qkv_fn): Sequential(
            (0): Linear(in_features=128, out_features=24576, bias=False)
            (1): Rearrange('b n (qkv h d) -> qkv b h n d', qkv=3, h=8)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (fc): Linear(in_features=8192, out_features=128, bias=True)
        )
        (2): Residual()
      )
     

In [8]:
from text_recognizer.networks.transformer.nystromer.nystromer import Nystromer

In [9]:
efficient_transformer = Nystromer(
    dim = 128,
    depth = 4,
    num_heads = 8,
    num_landmarks = 128
)

In [10]:
from text_recognizer.networks.transformer.vit import ViT

In [11]:
v = ViT(
    dim = 128,
    image_size = (576, 640),
    patch_size = (32, 32),
    transformer = efficient_transformer
).cuda()

In [12]:
t = torch.randn(16, 1, 576, 640).cuda()

In [13]:
o = v(t)

In [22]:
caption = torch.randint(0, 90, (16, 690)).cuda()

In [23]:
o.shape

torch.Size([16, 360, 128])

In [24]:
caption.shape

torch.Size([16, 690])

In [29]:
transformer_decoder(caption, context = o) # (1, 1024, 20000)

TypeError: forward() missing 2 required positional arguments: 'context' and 'context_mask'

In [ ]:
from text_recognizer.networks.encoders.efficientnet import EfficientNet

In [ ]:
en = EfficientNet()

In [ ]:
en.cuda()

In [ ]:
summary(en, (1, 576, 640))

In [ ]:
type(efficient_transformer)

In [ ]:
efficient_transformer = efficient_transformer(num_landmarks=256)

In [ ]:
efficient_transformer()

In [ ]:
from omegaconf import OmegaConf

In [ ]:
path = "../training/configs/vqvae.yaml"

In [ ]:
conf = OmegaConf.load(path)

In [ ]:
print(OmegaConf.to_yaml(conf))

In [ ]:
from text_recognizer.networks import VQVAE

In [ ]:
vae = VQVAE(**conf.network.args)

In [ ]:
vae

In [ ]:
datum = torch.randn([2, 1, 576, 640])

In [ ]:
vae.encoder(datum)[0].shape

In [ ]:
vae(datum)[0].shape

In [ ]:
datum = torch.randn([2, 1, 576, 640])

In [ ]:
trg = torch.randint(0, 1000, [2, 682])

In [ ]:
trg.shape

In [ ]:
datum = torch.randn([2, 1, 224, 224])

In [ ]:
en(t).shape

In [ ]:
path = "../training/configs/cnn_transformer.yaml"

In [ ]:
conf = OmegaConf.load(path)

In [ ]:
print(OmegaConf.to_yaml(conf))

In [ ]:
from text_recognizer.networks.cnn_transformer import CNNTransformer

In [ ]:
t = CNNTransformer(input_shape=(1, 576, 640), output_shape=(682, 1), **conf.network.args)

In [ ]:
t.encode(datum).shape

In [ ]:
trg.shape

In [ ]:
t(datum, trg).shape

In [ ]:
b, n = 16, 128
device = "cpu"

In [ ]:
x = lambda: torch.ones((b, n), device=device).bool()

In [ ]:
x().shape

In [ ]:
torch.ones((b, n), device=device).bool().shape

In [ ]:
x = torch.randn(1, 1, 576, 640)

In [ ]:
576 // 32

In [ ]:
640 // 32

In [ ]:
18 * 20

In [ ]:
x = torch.randn(1, 1, 144, 160)

In [ ]:
from einops import rearrange

In [ ]:
patch_size=16
p = rearrange(x, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size)

In [ ]:
p.shape